In [1]:
import matplotlib.pyplot as plt
import numpy as np
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense,Activation,Dropout, Flatten
import seaborn as sns
import os
import cv2
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
import pickle
from keras.models import model_from_json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')


Using TensorFlow backend.
c:\users\asus\python376\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\asus\python376\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\asus\python376\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\asus\python376\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [2]:
path = r"augmented_images"
model_folder = "Model2"
#This will be used as the folder name where the model will be stored.
categories = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
categories

['0', '1']

In [3]:
X_file = os.path.join(model_folder, "X.txt.npy") ## Construct file paths for loading and saving NumPy arrays
Y_file = os.path.join(model_folder, "Y.txt.npy")
if os.path.exists(X_file) and os.path.exists(Y_file): ## Check if the files alreadyexist
    X = np.load(X_file)
    Y = np.load(Y_file) ## Load the arrays from the files
    print("X and Y arrays loaded successfully.")
else:
    ## Initialize empty arrays for input and output
    X = [] # input array
    Y = [] # output array
    # Traverse through the directory specified by 'path'
    for root, dirs, directory in os.walk(path):
    # Loop through the files in the directory
        for j in range(len(directory)):
            name = os.path.basename(root)      #extract the category name from the directory path
            print(f'Loading category: {dirs}')     # Print the category being loaded
            print(name+" "+root+"/"+directory[j])    # Print the path of the current image being loaded
            if 'Thumbs.db' not in directory[j]:     ## Check if the current file is not 'Thumbs.db'
                img_array = cv2.imread(root+"/"+directory[j])   ## Read the image fileusing OpenCV
                img_resized = cv2.resize(img_array, (64,64))   # Resize the image to 64x64 pixels
                im2arr = np.array(img_resized)      # Convert the resized image to a NumPy array
                im2arr = im2arr.reshape(64,64,3)    # Reshape the array to match theexpected input shape (64x64x3)
                X.append(im2arr)       # Append the index of the category in categories list to Y
                Y.append(categories.index(name)) # Append the index of the category incategories list to the output array (Y)
    X = np.asarray(X)        ## Convert the lists to NumPy arrays
    Y = np.asarray(Y)
    X = X.astype('float32')      # Convert pixel values to float32 and normalize them to arange between 0 and 1
    X = X / 255               # Normalize pixel values
    Y = to_categorical(Y, num_classes=len(categories))         # Convert labels to one-hot encoding
## Save the processed arrays to files
    np.save(X_file, X)
    np.save(Y_file, Y)
    print("X and Y arrays saved successfully.")
# Shuffle the data using randomly generated indices
indices = np.arange(X.shape[0]) # it give shape of that matrix
np.random.shuffle(indices)
X = X[indices]
Y = Y[indices]


X and Y arrays loaded successfully.


In [4]:
num_classes = len(categories)
num_classes


2

In [5]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Example class labels, modify as necessary
labels = ['0', '1']  
accuracy, precision, recall, fscore = [], [], [], []  # Lists to store metrics

def calculateMetrics(algorithm, predict, testY, labels):
    testY = testY.astype('int')
    predict = predict.astype('int')

    # Calculate metrics
    a = accuracy_score(testY, predict) * 100
    p = precision_score(testY, predict, average='macro') * 100
    r = recall_score(testY, predict, average='macro') * 100
    f = f1_score(testY, predict, average='macro') * 100

    # Append metrics to lists
    accuracy.append(a)
    precision.append(p)
    recall.append(r)
    fscore.append(f)

    # Print metrics
    print(f"{algorithm} Accuracy: {a:.2f}")
    print(f"{algorithm} Precision: {p:.2f}")
    print(f"{algorithm} Recall: {r:.2f}")
    print(f"{algorithm} F1 Score: {f:.2f}")
    print(f"")
    print(f"")

    # Classification report
    report = classification_report(testY, predict, target_names=labels, output_dict=True)
    conf_matrix = confusion_matrix(testY, predict)
   # Compute and print per-class metrics
    print(f"{algorithm} Classification Report:")
    print(f"")

    class_accuracy = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
    print(f"{'Class':<10}{'Accuracy':<10}{'Precision':<10}{'Recall':<10}{'F1 Score':<10}")
    for i, class_name in enumerate(labels):
        print(f"{class_name:<10}{class_accuracy[i]:<10.4f}{report[class_name]['precision']:<10.4f}{report[class_name]['recall']:<10.4f}{report[class_name]['f1-score']:<10.4f}")
    
    # Confusion matrix
    plt.figure(figsize=(5, 5))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels)
    plt.title(f"{algorithm} Confusion Matrix")
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

 


In [7]:
# Check if the pkl file exists
Model_file = os.path.join(model_folder, "DLmodel.json") # Check if the JSON filefor the model exists

## Constructs the file path for the model's JSON file within the specified model folder.
Model_weights = os.path.join(model_folder, "DLmodel_weights.h5") #weights filewithin the specified model folder.

Model_history = os.path.join(model_folder, "history.pckl") # Constructs the file path for the model's training history file within the specified model folder.
if os.path.exists(Model_file):
        # Checks if the JSON file for the model exists.
    with open(Model_file, "r") as json_file:
    # Opens the JSON file for reading.
        loaded_model_json = json_file.read()
    # Reads the contents of the JSON file into a string.
        model = model_from_json(loaded_model_json)
    # Constructs a model from the JSON string representation.
    json_file.close()
    json_file.close()
    model.load_weights(Model_weights)
        ## Loads the weights of the model from the specified weights file.
    model._make_predict_function()
    print(model.summary())
    f = open(Model_history, 'rb') # Opens the training history file for reading in binary mode.
    accuracy = pickle.load(f) # Loads the training history dictionary from the pickle file.
    f.close() # Closes the history file.
    acc = accuracy['accuracy']
    acc = acc[9] * 100 # Selects the accuracy value corresponding to the 10thepoch and converts it to a percentage.
    print("CNN Model Prediction Accuracy = " + str(acc))
else:
    # If the model files do not exist, create and train a new model.
    model = Sequential() #resnet transfer learning code here
    model.add(Convolution2D(32, 3, 3, input_shape = (64, 64, 3), activation = 'relu'))
    # Adds a 2D convolutional layer with 32 filters, a 3x3 kernel size, and ReLU activation.

    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Convolution2D(32, 3, 3, activation = 'relu'))
    model.add(MaxPooling2D(pool_size = (2, 2)))
    model.add(Flatten())
    model.add(Dense(output_dim = 256, activation = 'relu'))
    # Adds a fully connected layer with 256 neurons and ReLU activation.
    model.add(Dense(output_dim = num_classes, activation = 'softmax')) # optimizer use is minimizing the loss function
    # Adds an output layer with 'num_classes' neurons and softmax activation.
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    print(model.summary())
    #hist = model.fit(X, Y, batch_size=16, epochs=10, validation_split=0.2, shuffle=True,verbose=2)
    hist = model.fit(X_train, Y_train, batch_size=16, epochs=50, validation_data=(X_test, Y_test), shuffle=True, verbose=2)
    # Trains the model with the specified training data for 25 epochs, with batch size 16, and validates on test data
    model.save_weights(Model_weights) # Saves the trained model weights
    model_json = model.to_json() ## Converts the model to JSON format.
    with open(Model_file, "w") as json_file:
    # Opens the JSON file for writing.
        json_file.write(model_json)
    json_file.close()
    f = open(Model_history, 'wb')
    pickle.dump(hist.history, f)
    f.close()
    f = open(Model_history, 'rb') # Opens the history file for writing in binary mode.
    accuracy = pickle.load(f) # Saves the training history dictionary to the file

    f.close()
    acc = accuracy['accuracy']
    acc = acc[9] * 100
    print("CNN Model Prediction Accuracy = "+str(acc))



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               1605888   
_________________________________________________________________
dense_2 (Dense)              (None, 2)              

In [8]:
import numpy as np

# Generate predictions and convert them to class labels
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# If y_test and y_pred_classes are one-hot encoded, convert them to class labels
if Y_test.ndim > 1 and Y_test.shape[1] > 1:
    Y_test = np.argmax(Y_test, axis=1)

# Define categories or use the previously defined labels
categories = ['0', '1']  # Adjust to match your actual class names

# Run the function to calculate metrics
calculateMetrics("CNN Model", y_pred_classes, Y_test, categories)


AttributeError: 'dict' object has no attribute 'append'

## test 

In [ ]:

path = r'testimges/1.jpg'
img = cv2.imread(path)
img_resized = cv2.resize(img, (64,64))    # every image in different size hence we resize the image.
im2arr = np.array(img_resized)           # Convert the resized image into a NumPy array
img_reshaped = im2arr.reshape(1,64,64,3)         ### Reshape the NumPy array to match the expected input shape of the model (1, 64, 64, 3)

test = np.asarray(img_reshaped)
# Convert the reshaped image array into a NumPy array

test = test.astype('float32')
test = test/255
pred_probability = model.predict(test)
pred_number = np.argmax(pred_probability)   #argmax is a fundamental function in deeplearning for

# converting probability distributions into class labels
## This returns the index of the maximum value in the pred_probability array,
# which corresponds to the predicted class label.

output_name=categories[pred_number] # Get the corresponding output class name basedon the index
plt.imshow(img)
plt.text(10, 10, f'Predicted Output: {output_name}', color='white', fontsize=12, weight='bold', backgroundcolor='black')
plt.axis('off')

plt.show()

In [ ]:

path = r'testimges/20.jpg'
img = cv2.imread(path)
img_resized = cv2.resize(img, (64,64))    # every image in different size hence we resize the image.
im2arr = np.array(img_resized)           # Convert the resized image into a NumPy array
img_reshaped = im2arr.reshape(1,64,64,3)         ### Reshape the NumPy array to match the expected input shape of the model (1, 64, 64, 3)

test = np.asarray(img_reshaped)
# Convert the reshaped image array into a NumPy array

test = test.astype('float32')
test = test/255
pred_probability = model.predict(test)
pred_number = np.argmax(pred_probability)   #argmax is a fundamental function in deeplearning for

# converting probability distributions into class labels
## This returns the index of the maximum value in the pred_probability array,
# which corresponds to the predicted class label.

output_name=categories[pred_number] # Get the corresponding output class name basedon the index
plt.imshow(img)
plt.text(10, 10, f'Predicted Output: {output_name}', color='white', fontsize=12, weight='bold', backgroundcolor='black')
plt.axis('off')

plt.show()

In [ ]:

path = r'testimges/22.jpg'
img = cv2.imread(path)
img_resized = cv2.resize(img, (64,64))    # every image in different size hence we resize the image.
im2arr = np.array(img_resized)           # Convert the resized image into a NumPy array
img_reshaped = im2arr.reshape(1,64,64,3)         ### Reshape the NumPy array to match the expected input shape of the model (1, 64, 64, 3)

test = np.asarray(img_reshaped)
# Convert the reshaped image array into a NumPy array

test = test.astype('float32')
test = test/255
pred_probability = model.predict(test)
pred_number = np.argmax(pred_probability)   #argmax is a fundamental function in deeplearning for

# converting probability distributions into class labels
## This returns the index of the maximum value in the pred_probability array,
# which corresponds to the predicted class label.

output_name=categories[pred_number] # Get the corresponding output class name basedon the index
plt.imshow(img)
plt.text(10, 10, f'Predicted Output: {output_name}', color='white', fontsize=12, weight='bold', backgroundcolor='black')
plt.axis('off')

plt.show()

In [ ]:

path = r'testimges/17.jpg'
img = cv2.imread(path)
img_resized = cv2.resize(img, (64,64))    # every image in different size hence we resize the image.
im2arr = np.array(img_resized)           # Convert the resized image into a NumPy array
img_reshaped = im2arr.reshape(1,64,64,3)         ### Reshape the NumPy array to match the expected input shape of the model (1, 64, 64, 3)

test = np.asarray(img_reshaped)
# Convert the reshaped image array into a NumPy array

test = test.astype('float32')
test = test/255
pred_probability = model.predict(test)
pred_number = np.argmax(pred_probability)   #argmax is a fundamental function in deeplearning for

# converting probability distributions into class labels
## This returns the index of the maximum value in the pred_probability array,
# which corresponds to the predicted class label.

output_name=categories[pred_number] # Get the corresponding output class name basedon the index
plt.imshow(img)
plt.text(10, 10, f'Predicted Output: {output_name}', color='white', fontsize=12, weight='bold', backgroundcolor='black')
plt.axis('off')

plt.show()

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set image dimensions
img_width, img_height = 64, 64
input_shape = (img_width, img_height, 3)

# Load and preprocess image data
data_dir =r"Dataset/"  # Replace with your dataset path

# Use ImageDataGenerator to load and preprocess images
datagen = ImageDataGenerator(rescale=1.0/255)
train_generator = datagen.flow_from_directory(
    directory=os.path.join(data_dir, 'train'),
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_generator = datagen.flow_from_directory(
    directory=os.path.join(data_dir, 'test'),
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

# Extract features and labels from generators
x_train, y_train = next(train_generator)
x_test, y_test = next(test_generator)

# Build a CNN model for feature extraction
def build_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    return model

# Initialize CNN model
cnn_model = build_cnn_model(input_shape)

# Extract features using the CNN model (ignore the final dense layer for feature extraction)
cnn_feature_extractor = Sequential(cnn_model.layers[:-1])  # Exclude the final dense layer

# Extract features from CNN model
x_train_features = cnn_feature_extractor.predict(x_train)
x_test_features = cnn_feature_extractor.predict(x_test)

# Define the path to save/load the hybrid model
hybrid_model_path = 'model/hybrid_model_image.npy'

# Train CatBoost if the model is not already saved
if os.path.exists(hybrid_model_path):
    hybrid_model = CatBoostClassifier().load_model(hybrid_model_path)
else:
    hybrid_model = CatBoostClassifier()
    hybrid_model.fit(x_train_features, y_train, verbose=False)
    hybrid_model.save_model(hybrid_model_path)

# Predict and evaluate the hybrid model
y_pred_hybrid = hybrid_model.predict(x_test_features)
hybrid_model_accuracy = accuracy_score(y_test, y_pred_hybrid)

print("Hybrid Model (Image) Accuracy:", hybrid_model_accuracy)


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Define the image path and categories
path = r'testimges/17.jpg'
categories = ['0', '1']  # Replace with your actual class names

# Load and preprocess the image
img = cv2.imread(path)

# Check if the image was loaded successfully
if img is None:
    print("Error: Image not found or could not be loaded. Check the file path.")
else:
    # Proceed only if the image was loaded correctly
    img_resized = cv2.resize(img, (64, 64))  # Resize the image to match the input shape
    img_array = np.array(img_resized)        # Convert the resized image into a NumPy array
    img_reshaped = img_array.reshape(1, 64, 64, 3)  # Reshape to match the CNN input shape

    # Normalize the image data
    test_img = img_reshaped.astype('float32') / 255.0

    # Extract features using the CNN model (cnn_feature_extractor is used for feature extraction)
    test_features = cnn_feature_extractor.predict(test_img)

    # Predict using CatBoost with extracted features
    pred_probability = hybrid_model.predict_proba(test_features)
    pred_number = np.argmax(pred_probability)  # Get the class with the highest probability

    # Get the predicted class label
    output_name = categories[pred_number]

    # Display the image and prediction
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert color for correct display
    plt.text(10, 10, f'Predicted Output: {output_name}', color='white', fontsize=12, weight='bold', backgroundcolor='black')
    plt.axis('off')
    plt.show()
